In [528]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [529]:
data = pd.read_csv('train.csv')
X = data.iloc[:,[2,4,5,6,7,9,11]]
y_train = data.iloc[:,1]
test_data = pd.read_csv('test.csv')
X_test = test_data.iloc[:,[1,3,4,5,6,8,10]]

In [530]:
categorical_features = X.select_dtypes(include=['object','category']).columns
imputer = SimpleImputer(strategy='constant',fill_value='Missing')
X.loc[:,categorical_features] = imputer.fit_transform(X[categorical_features])
X_test.loc[:,categorical_features] = imputer.transform(X_test[categorical_features])



In [531]:
imputer = SimpleImputer(strategy='median')
X.loc[:,'Age'] = imputer.fit_transform(X[['Age']])
X_test.loc[:,'Age'] = imputer.transform(X_test[['Age']])
X.loc[:,'Fare'] = imputer.fit_transform(X[['Fare']])
X_test.loc[:,'Fare'] = imputer.transform(X_test[['Fare']])



In [532]:
X = X.copy()
X_test = X_test.copy()
X.loc[:,'FamilySize'] = X.loc[:,'SibSp'] + X.loc[:,'Parch'] + 1
X.loc[:,'IsAlone'] = (X.loc[:,'FamilySize']==1).astype(float)
X_test.loc[:,'FamilySize'] = X_test.loc[:,'SibSp'] + X_test.loc[:,'Parch'] + 1
X_test.loc[:,'IsAlone'] = (X_test.loc[:,'FamilySize']==1).astype(float)

In [533]:
X = pd.get_dummies(X,columns=['Sex','Embarked']).astype(float)
X_test = pd.get_dummies(X_test,columns=['Sex','Embarked']).astype(float)



In [534]:
X_test = X_test.reindex(columns=X.columns,fill_value=0)

In [535]:
scalar = StandardScaler()
numerical_features = ['Age', 'Fare', 'SibSp', 'Parch', 'FamilySize']
X.loc[:,numerical_features] = scalar.fit_transform(X[numerical_features])
X_test.loc[:,numerical_features] = scalar.transform(X_test[numerical_features])

In [544]:
X_train, X_val, y_train, y_val = train_test_split(X,y_train,test_size=0.2,random_state=100)

In [545]:
reg = LogisticRegression(C=1.0, random_state=100)

In [546]:
reg.fit(X_train,y_train)

LogisticRegression(random_state=100)

In [547]:
y_pred_val = reg.predict(X_val)

In [548]:
accuracy = accuracy_score(y_val,y_pred_val)

In [549]:
accuracy

0.8044692737430168

In [550]:
y_pred = reg.predict(X_test)

In [551]:
result = pd.DataFrame(
    {
        "PassengerId":list(range(892,892+len(y_pred))),
        "Survived" : y_pred
    }
    )
result.to_csv("submission.csv",index=False)